# coletando séries:

## Parte 1
- Faça scraping para obter os 250 séries com as maiores avaliações do IMDB. Devem ser
obtidos: Título, Ano de estreia e encerramento (se existir), Número Total de Episódios,
Classificação Indicativa, Nota do IMDB e o link para página da série.

In [72]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.imdb.com")
driver.maximize_window()

# Abre o menu
menu = driver.find_element(By.XPATH, r"//*[@id='imdbHeader-navDrawerOpen']")
menu.click()

# Dá um tempo para o menu abrir e o DOM atualizar
time.sleep(1)  

# Busca o link de "As 250 séries mais bem avaliadas" depois que o menu abriu
link_populares = driver.find_element(
    By.XPATH, r"//*[@id='imdbHeader']/div/aside[1]/div/div[2]/div/div[2]/div[1]/span/div/div/ul/a[2]"
)

# Pega o href
url_series = link_populares.get_attribute("href")
print(url_series)

# Abrir a página 
driver.get(url_series)


https://www.imdb.com/pt/chart/toptv/?ref_=hm_nv_menu


In [73]:
# pega a lista (ul) que contém todas as séries
tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')

# pega todas as séries (li dentro da ul)
lista_series = tag_ul.find_elements(By.TAG_NAME, "li")

print(type(tag_ul)) # WebElement pra dar certo
print(type(lista_series))    
print(len(lista_series))     


<class 'selenium.webdriver.remote.webelement.WebElement'>
<class 'list'>
250


In [74]:
dados_series = []
for serie in lista_series:
    titulo = serie.find_element(By.TAG_NAME, "h3").text

    # pega todos os spans de metadados
    spans = serie.find_elements(By.CLASS_NAME, "cli-title-metadata-item")
    anos, episodios, classificacao = None, None, None

    if len(spans) > 0:
        anos = spans[0].text.strip()  # ex: "2008–2013"
    if len(spans) > 1:
        episodios = spans[1].text.strip()  # ex: "62 episódios"
    if len(spans) > 2:
        classificacao = spans[2].text.strip()  # ex: "16"

    nota = serie.find_element(By.CLASS_NAME, "ipc-rating-star--rating").text
    link = serie.find_element(By.TAG_NAME, "a").get_attribute("href")

    dados_series.append({
        "titulo": titulo,
        "anos": anos,
        "episodios": episodios,
        "classificacao": classificacao,
        "nota": float(nota.replace(",", ".")),
        "link": link
    })
print(dados_series[:3])  # Conferir os 3 primeiros


[{'titulo': '1. Breaking Bad', 'anos': '2008–2013', 'episodios': '62 episódios', 'classificacao': '16', 'nota': 9.5, 'link': 'https://www.imdb.com/pt/title/tt0903747/?ref_=chttvtp_i_1'}, {'titulo': '2. Planeta Terra II', 'anos': '2016', 'episodios': '6 episódios', 'classificacao': '10', 'nota': 9.4, 'link': 'https://www.imdb.com/pt/title/tt5491994/?ref_=chttvtp_i_2'}, {'titulo': '3. Planeta Terra', 'anos': '2006', 'episodios': '11 episódios', 'classificacao': 'Livre', 'nota': 9.4, 'link': 'https://www.imdb.com/pt/title/tt0795176/?ref_=chttvtp_i_3'}]


## Parte 2:
- Faça scraping das páginas específicas dos 250 séries obtidos no item anterior. Obte-
nha dessa página a popularidade e a listagem do elenco principal (incluindo nome do
ator/atriz e da personagem e número de episódios que participou).

In [75]:
import json
import time
from selenium.webdriver.common.by import By
import re

detalhes_series = []

for serie in dados_series:  
    try:
        driver.get(serie["link"])
        time.sleep(1.5)

        # Popularidade 
        try:
            popularidade = driver.find_element(
                By.XPATH, "//div[contains(@data-testid,'hero-rating-bar__popularity__score')]"
            ).text.strip()
            popularidade = int(popularidade)
        except:
            popularidade = None

        # Elenco principal 
        elenco = []
        try:
            elenco_blocos = driver.find_elements(By.XPATH, "//div[@data-testid='title-cast-item']")
            for bloco in elenco_blocos:
                # Ator
                try:
                    ator = bloco.find_element(By.XPATH, ".//a[@data-testid='title-cast-item__actor']").text.strip()
                except:
                    ator = None

                # Personagem
                try:
                    personagem = bloco.find_element(By.XPATH, ".//a[@data-testid='cast-item-characters-link']/span").text.strip()
                except:
                    personagem = None

                # Episódios
                try:
                    botoes_eps = bloco.find_elements(By.XPATH, ".//button[contains(@data-testid,'eps-toggle')]")
                    episodios_texto = botoes_eps[-1].text.strip() if botoes_eps else None
                    episodios = int(re.search(r'\d+', episodios_texto).group()) if episodios_texto else None
                except:
                    episodios = None

                elenco.append({
                    "ator": ator,
                    "personagem": personagem,
                    "episodios": episodios
                })
        except:
            elenco = []

        # Transformações título / classificacao / episodios
        titulo = re.sub(r'^\d+\.\s*', '', serie["titulo"])
        try:
            classificacao = int(serie["classificacao"])
        except:
            classificacao = serie["classificacao"]

        try:
            episodios_serie = int(re.search(r'\d+', serie["episodios"]).group())
        except:
            episodios_serie = None

        serie_detalhes = {
            "titulo": titulo,
            "anos": serie["anos"],
            "episodios": episodios_serie,
            "classificacao": classificacao,
            "nota": float(serie["nota"]),
            "link": serie["link"],
            "popularidade": popularidade,
            "elenco_principal": elenco
        }

        detalhes_series.append(serie_detalhes)

    except Exception as e:
        print(f"Erro ao processar {serie['titulo']}: {e}")
        continue

print(detalhes_series[0])

{'titulo': 'Breaking Bad', 'anos': '2008–2013', 'episodios': 62, 'classificacao': 16, 'nota': 9.5, 'link': 'https://www.imdb.com/pt/title/tt0903747/?ref_=chttvtp_i_1', 'popularidade': 18, 'elenco_principal': [{'ator': 'Bryan Cranston', 'personagem': 'Walter White', 'episodios': 62}, {'ator': 'Aaron Paul', 'personagem': 'Jesse Pinkman', 'episodios': 62}, {'ator': 'Anna Gunn', 'personagem': 'Skyler White', 'episodios': 62}, {'ator': 'Betsy Brandt', 'personagem': 'Marie Schrader', 'episodios': 62}, {'ator': 'RJ Mitte', 'personagem': 'Walter White, Jr.', 'episodios': 62}, {'ator': 'Dean Norris', 'personagem': 'Hank Schrader', 'episodios': 62}, {'ator': 'Bob Odenkirk', 'personagem': 'Saul Goodman', 'episodios': 43}, {'ator': 'Steven Michael Quezada', 'personagem': 'Steven Gomez', 'episodios': 34}, {'ator': 'Jonathan Banks', 'personagem': 'Mike Ehrmantraut', 'episodios': 28}, {'ator': 'Giancarlo Esposito', 'personagem': 'Gus Fring', 'episodios': 26}, {'ator': 'Charles Baker', 'personagem': '

## Parte 3:
- Salve as informações obtidas em um arquivo json. Tome cuidado para aplicar as trans-
formações necessárias nos dados extraídos (remover números do título, conversão de
números inteiros e reais para os devidos tipos).

In [76]:
with open("data/series_detalhes.json", "w", encoding="utf-8") as f:
    json.dump(detalhes_series, f, ensure_ascii=False, indent=4)
